In [1]:
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1A/blp25_hatespeech_subtask_1A_train.tsv
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1A/blp25_hatespeech_subtask_1A_dev.tsv
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1A/blp25_hatespeech_subtask_1A_dev_test.tsv

--2025-08-20 16:56:20--  https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1A/blp25_hatespeech_subtask_1A_train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8002036 (7.6M) [text/plain]
Saving to: ‘blp25_hatespeech_subtask_1A_train.tsv.4’

blp25_hatespeech_su 100%[===================>]   7.63M  --.-KB/s    in 0.06s   

2025-08-20 16:56:20 (134 MB/s) - ‘blp25_hatespeech_subtask_1A_train.tsv.4’ saved [8002036/8002036]

--2025-08-20 16:56:21--  https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1A/blp25_hatespeech_subtask_1A_dev.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent

In [2]:
import logging
import os
import random
import sys
from dataclasses import dataclass, field
from typing import Optional
import pandas as pd
import datasets
import evaluate
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict
import torch

import transformers
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version


logger = logging.getLogger(__name__)

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)

In [34]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./Ensamble_HateSpeech/",
    
    # Your core changes
    num_train_epochs=5,  # Increased from 1 to 3. You can experiment with 4 or 5.
    
    # Important adjustments for finding the best model
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",        # Save a checkpoint at the end of each epoch
    load_best_model_at_end=True,  # This will load the best checkpoint when training is done
    
    # Other common parameters
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    save_total_limit=2,           # Saves disk space by only keeping the best 2 checkpoints
    report_to="none",
    overwrite_output_dir=True,
)

max_train_samples = None
max_eval_samples=None
max_predict_samples=None
max_seq_length = 512
batch_size = 16

/users/t/p/tprama/.local/lib/python3.12/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
[INFO|training_args.py:2183] 2025-08-20 17:35:26,226 >> PyTorch: setting up devices


In [35]:
transformers.utils.logging.set_verbosity_info()

log_level = training_args.get_process_log_level()
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
    + f" distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
)
logger.info(f"Training/evaluation parameters {training_args}")

08/20/2025 17:35:26 - WARNING - __main__ - Process rank: 0, device: cpu, n_gpu: 0 distributed training: True, 16-bits training: False
08/20/2025 17:35:26 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=0,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=Non

In [36]:
train_file = 'blp25_hatespeech_subtask_1A_train.tsv'
validation_file = 'blp25_hatespeech_subtask_1A_dev.tsv'
test_file = 'blp25_hatespeech_subtask_1A_dev_test.tsv'

In [37]:
model_name = 'mDeBERTa‑v3‑base'

#model_name = "csebuetnlp/banglabert" 

In [38]:
l2id = {'None': 0, 'Religious Hate': 1, 'Sexism': 2, 'Political Hate': 3, 'Profane': 4, 'Abusive': 5}
train_df = pd.read_csv(train_file, sep='\t')
# print(train_df['label'])
train_df['label'] = train_df['label'].map(l2id).fillna(0).astype(int)
train_df = Dataset.from_pandas(train_df)
validation_df = pd.read_csv(validation_file, sep='\t')
validation_df['label'] = validation_df['label'].map(l2id).fillna(0).astype(int)
validation_df = Dataset.from_pandas(validation_df)
test_df = pd.read_csv(test_file, sep='\t')
#test_df['label'] = test_df['label'].map(l2id)
test_df = Dataset.from_pandas(test_df)

data_files = {"train": train_df, "validation": validation_df, "test": test_df}
for key in data_files.keys():
    logger.info(f"loading a local file for {key}")
raw_datasets = DatasetDict(
    {"train": train_df, "validation": validation_df, "test": test_df}
)

train_df

08/20/2025 17:35:27 - INFO - __main__ - loading a local file for train
08/20/2025 17:35:27 - INFO - __main__ - loading a local file for validation
08/20/2025 17:35:27 - INFO - __main__ - loading a local file for test


Dataset({
    features: ['id', 'text', 'label'],
    num_rows: 35522
})

In [39]:
import pandas as pd

# convert first 10 rows of the "train" split into a pandas DataFrame
df = raw_datasets["train"].to_pandas()
print(df.head(10))


       id                                               text  label
0  147963  ধন্যবাদ বর্ডার গার্ড দেরকে এভাবে পাহারা দিতে হ...      0
1  214275  ছোটবেলায় অনেক কষ্ট করে কিছু গালাগালি শিখছিলাম...      0
2  849172          অতিরিক্ত এ নিজেকে বাদুর বানাইয়া ফেলছেন রে      5
3  821985  চিন ভারত রাশিয়া এই তিন দেশ এক থাকলে বিশ্বকে শা...      0
4  477288  এটার বিচার কে করবেযে বিচার করবে সেই তো হলো এই ...      5
5  933728    তুরা কিসের জন্য দুভাই যাবি অন্য দেশে কেন জাস না      0
6  398351  দেশ বিভাগের সময়ে পশ্চিম পাকিস্তানে ২৫ শতাংশ স...      0
7  786609  ইরান পারমাণবিক বোমা বানাবে বানাবে বলতে বলতে বি...      5
8  917115                                  আজকে এই উৎসব কেনো      0
9  415453  ইমরান ছাড়া পাকিস্তান কখনোই ঘুরে দাড়াতে পারবে ন...      0


In [40]:
transformers.utils.logging.set_verbosity_info()

log_level = training_args.get_process_log_level()
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
    + f" distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
)
logger.info(f"Training/evaluation parameters {training_args}")

08/20/2025 17:35:28 - WARNING - __main__ - Process rank: 0, device: cpu, n_gpu: 0 distributed training: True, 16-bits training: False
08/20/2025 17:35:28 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=0,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=Non

In [41]:
# Labels
label_list = raw_datasets["train"].unique("label")
print(label_list)
label_list.sort()  # sort the labels for determine
num_labels = len(label_list)

[0, 5, 4, 1, 3, 2]


In [42]:

from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification

# Define your model name and number of labels
model_name = "microsoft/deberta-v3-base"
num_labels = 6 # Based on your l2id dictionary: None, Religious, Sexism, etc.

# Load the model configuration
config = AutoConfig.from_pretrained(
    model_name,
    num_labels=num_labels,
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
)

# Load the model FOR SEQUENCE CLASSIFICATION
# This is the key change!
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    config=config,
    ignore_mismatched_sizes=True, # Important when adding a new head
)

[INFO|configuration_utils.py:699] 2025-08-20 17:35:29,409 >> loading configuration file config.json from cache at /users/t/p/tprama/.cache/huggingface/hub/models--microsoft--deberta-v3-base/snapshots/8ccc9b6f36199bec6961081d44eb72fb3f7353f3/config.json
[INFO|configuration_utils.py:771] 2025-08-20 17:35:29,409 >> Model config DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-base",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-07,
  "legacy": true,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_

In [43]:
non_label_column_names = [name for name in raw_datasets["train"].column_names if name != "label"]
sentence1_key= non_label_column_names[1]

# Padding strategy
padding = "max_length"

# Some models have set the order of the labels to use, so let's make sure we do use it.
label_to_id = None
if (model.config.label2id != PretrainedConfig(num_labels=num_labels).label2id):
    # Some have all caps in their config, some don't.
    label_name_to_id = {k.lower(): v for k, v in model.config.label2id.items()}
    if sorted(label_name_to_id.keys()) == sorted(label_list):
        label_to_id = {i: int(label_name_to_id[label_list[i]]) for i in range(num_labels)}
    else:
        logger.warning(
            "Your model seems to have been trained with labels, but they don't match the dataset: ",
            f"model labels: {sorted(label_name_to_id.keys())}, dataset labels: {sorted(label_list)}."
            "\nIgnoring the model labels as a result.",)

if label_to_id is not None:
    model.config.label2id = label_to_id
    model.config.id2label = {id: label for label, id in config.label2id.items()}

if 128 > tokenizer.model_max_length:
    logger.warning(
        f"The max_seq_length passed ({128}) is larger than the maximum length for the"
        f"model ({tokenizer.model_max_length}). Using max_seq_length={tokenizer.model_max_length}.")
max_seq_length = min(128, tokenizer.model_max_length)

def preprocess_function(examples):
    # Tokenize the texts
    args = (
        (examples[sentence1_key],))
    result = tokenizer(*args, padding=padding, max_length=max_seq_length, truncation=True)

    # Map labels to IDs (not necessary for GLUE tasks)
    if label_to_id is not None and "label" in examples:
        result["label"] = [(label_to_id[l] if l != -1 else -1) for l in examples["label"]]
    return result
raw_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    load_from_cache_file=True,
    desc="Running tokenizer on dataset",
)


Running tokenizer on dataset:   0%|          | 0/35522 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/2512 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/2512 [00:00<?, ? examples/s]

In [44]:
if "train" not in raw_datasets:
    raise ValueError("requires a train dataset")
train_dataset = raw_datasets["train"]
if max_train_samples is not None:
    max_train_samples_n = min(len(train_dataset), max_train_samples)
    train_dataset = train_dataset.select(range(max_train_samples_n))

In [45]:
if "validation" not in raw_datasets:
    raise ValueError("requires a validation dataset")
eval_dataset = raw_datasets["validation"]
if max_eval_samples is not None:
    max_eval_samples_n = min(len(eval_dataset), max_eval_samples)
    eval_dataset = eval_dataset.select(range(max_eval_samples_n))

In [46]:
if "test" not in raw_datasets and "test_matched" not in raw_datasets:
    raise ValueError("requires a test dataset")
predict_dataset = raw_datasets["test"]
if max_predict_samples is not None:
    max_predict_samples_n = min(len(predict_dataset), max_predict_samples)
    predict_dataset = predict_dataset.select(range(max_predict_samples_n))

In [47]:
for index in random.sample(range(len(train_dataset)), 3):
    logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")

08/20/2025 17:35:33 - INFO - __main__ - Sample 7296 of the training set: {'id': 660, 'text': 'সরকারের দায়িত্বপ্রাপ্ত সংস্থা সকল বাণিজ্যিক ভবন গুলোকে বছরে অন্তত একবার পরিদর্শন করে রিপোর্ট প্রদান করবেন একটা বাণিজ্যিক ভবনে নিরাপত্তার সকল ব্যবস্থা থাকা উচিত কিন্তু এই সব সম্ভবের দেশে কিছুই করা হয় না এই দেশে মানুষের জীবনের কোন দাম নেই তাই কেউ কোনকিছুর তদারকিরও দরকার মনে করে না', 'label': 3, 'input_ids': [1, 119491, 84555, 67300, 105927, 53401, 84555, 507, 103135, 42055, 107186, 56829, 82319, 70385, 69119, 91091, 70385, 84555, 42055, 91091, 70385, 82319, 119491, 123544, 115329, 70385, 123207, 42055, 119491, 67300, 84144, 507, 69119, 42055, 123974, 56829, 116962, 70385, 123528, 56829, 67300, 507, 123047, 69119, 57264, 507, 122570, 101628, 84144, 113711, 67300, 53401, 507, 69119, 122838, 84555, 53401, 507, 123179, 57264, 70385, 82319, 82319, 507, 122630, 67300, 69119, 105927, 507, 91091, 84555, 56829, 103135, 84555, 70385, 122534, 57264, 507, 67300, 84555, 53401, 507, 84555, 56829, 91091, 11371

In [48]:
metric = evaluate.load("accuracy")

In [49]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}


In [50]:
data_collator = default_data_collator

In [51]:
train_dataset = train_dataset.remove_columns("id")
eval_dataset = eval_dataset.remove_columns("id")

In [52]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/tmp/ipykernel_1908056/1049306949.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [53]:
train_result = trainer.train()
metrics = train_result.metrics
max_train_samples = (
    max_train_samples if max_train_samples is not None else len(train_dataset)
)
metrics["train_samples"] = min(max_train_samples, len(train_dataset))



[INFO|trainer.py:928] 2025-08-20 17:35:35,246 >> The following columns in the training set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
[INFO|trainer.py:2405] 2025-08-20 17:35:35,252 >> ***** Running training *****
[INFO|trainer.py:2406] 2025-08-20 17:35:35,252 >>   Num examples = 35,522
[INFO|trainer.py:2407] 2025-08-20 17:35:35,252 >>   Num Epochs = 5
[INFO|trainer.py:2408] 2025-08-20 17:35:35,252 >>   Instantaneous batch size per device = 16
[INFO|trainer.py:2411] 2025-08-20 17:35:35,252 >>   Total train batch size (w. parallel, distributed & accumulation) = 16
[INFO|trainer.py:2412] 2025-08-20 17:35:35,253 >>   Gradient Accumulation steps = 1
[INFO|trainer.py:2413] 2025-08-20 17:35:35,253 >>   Total optimization steps = 11,105
[INFO|trainer.py:2414] 2025-08-20 17:35:35,253 >>   Number of trainable paramete

Epoch,Training Loss,Validation Loss,Accuracy
1,0.930600,0.846010,0.685510
2,0.842000,0.809374,0.696656
3,0.791700,0.787678,0.704220
4,0.752900,0.761758,0.704618
5,0.703300,0.763972,0.709793


[INFO|trainer.py:928] 2025-08-20 18:07:20,310 >> The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
[INFO|trainer.py:4258] 2025-08-20 18:07:20,313 >> 
***** Running Evaluation *****
[INFO|trainer.py:4260] 2025-08-20 18:07:20,314 >>   Num examples = 2512
[INFO|trainer.py:4263] 2025-08-20 18:07:20,314 >>   Batch size = 16
[INFO|trainer.py:3942] 2025-08-20 18:07:56,523 >> Saving model checkpoint to ./Ensamble_HateSpeech/checkpoint-2221
[INFO|configuration_utils.py:423] 2025-08-20 18:07:56,525 >> Configuration saved in ./Ensamble_HateSpeech/checkpoint-2221/config.json
[INFO|modeling_utils.py:3040] 2025-08-20 18:07:56,700 >> Model weights saved in ./Ensamble_HateSpeech/checkpoint-2221/model.safetensors
[INFO|tokenization_utils_base.py:2500] 2025-08-20 18:07:56,702 >> tokenizer 

In [54]:
trainer.save_model()
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

[INFO|trainer.py:3942] 2025-08-20 20:31:36,977 >> Saving model checkpoint to ./Ensamble_HateSpeech/
[INFO|configuration_utils.py:423] 2025-08-20 20:31:36,979 >> Configuration saved in ./Ensamble_HateSpeech/config.json
[INFO|modeling_utils.py:3040] 2025-08-20 20:31:37,143 >> Model weights saved in ./Ensamble_HateSpeech/model.safetensors
[INFO|tokenization_utils_base.py:2500] 2025-08-20 20:31:37,144 >> tokenizer config file saved in ./Ensamble_HateSpeech/tokenizer_config.json
[INFO|tokenization_utils_base.py:2509] 2025-08-20 20:31:37,144 >> Special tokens file saved in ./Ensamble_HateSpeech/special_tokens_map.json


***** train metrics *****
  epoch                    =        5.0
  total_flos               = 10881030GF
  train_loss               =     0.8249
  train_runtime            = 2:41:48.34
  train_samples            =      35522
  train_samples_per_second =     18.295
  train_steps_per_second   =      1.144


In [55]:
logger.info("*** Evaluate ***")

metrics = trainer.evaluate(eval_dataset=eval_dataset)

max_eval_samples = (
    max_eval_samples if max_eval_samples is not None else len(eval_dataset)
)
metrics["eval_samples"] = min(max_eval_samples, len(eval_dataset))

trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

08/20/2025 20:31:49 - INFO - __main__ - *** Evaluate ***


[INFO|trainer.py:928] 2025-08-20 20:31:49,043 >> The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
[INFO|trainer.py:4258] 2025-08-20 20:31:49,045 >> 
***** Running Evaluation *****
[INFO|trainer.py:4260] 2025-08-20 20:31:49,045 >>   Num examples = 2512
[INFO|trainer.py:4263] 2025-08-20 20:31:49,046 >>   Batch size = 16


***** eval metrics *****
  epoch                   =        5.0
  eval_accuracy           =     0.7046
  eval_loss               =     0.7618
  eval_runtime            = 0:00:36.31
  eval_samples            =       2512
  eval_samples_per_second =     69.175
  eval_steps_per_second   =      4.323


In [56]:
id2l = {v: k for k, v in l2id.items()}
logger.info("*** Predict ***")
#predict_dataset = predict_dataset.remove_columns("label")
ids = predict_dataset['id']
predict_dataset = predict_dataset.remove_columns("id")
predictions = trainer.predict(predict_dataset, metric_key_prefix="predict").predictions
predictions = np.argmax(predictions, axis=1)
output_predict_file = os.path.join(training_args.output_dir, f"subtask_1A.tsv")
if trainer.is_world_process_zero():
    with open(output_predict_file, "w") as writer:
        logger.info(f"***** Predict results *****")
        writer.write("id\tlabel\tmodel\n")
        for index, item in enumerate(predictions):
            item = label_list[item]
            item = id2l[item]
            writer.write(f"{ids[index]}\t{item}\t{model_name}\n")
     


08/20/2025 20:32:44 - INFO - __main__ - *** Predict ***


[INFO|trainer.py:928] 2025-08-20 20:32:44,095 >> The following columns in the test set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
[INFO|trainer.py:4258] 2025-08-20 20:32:44,096 >> 
***** Running Prediction *****
[INFO|trainer.py:4260] 2025-08-20 20:32:44,096 >>   Num examples = 2512
[INFO|trainer.py:4263] 2025-08-20 20:32:44,096 >>   Batch size = 16


08/20/2025 20:33:19 - INFO - __main__ - ***** Predict results *****


In [57]:
kwargs = {"finetuned_from": model_name, "tasks": "text-classification"}
trainer.create_model_card(**kwargs)

[INFO|modelcard.py:449] 2025-08-20 20:33:34,129 >> Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.7046178579330444}]}


In [58]:
import pandas as pd
from scipy.stats import mode

# Load predictions
bert = pd.read_csv("./BanglaBERT_HateSpeech/subtask_1A.tsv", sep="\t")
distil = pd.read_csv("./distilBERT_m/subtask_1A.tsv", sep="\t")
ens = pd.read_csv("./Ensamble_HateSpeech//subtask_1A.tsv", sep="\t")

# Assume each file has ["id","label"]
df = pd.DataFrame({
    "id": bert["id"],
    "bert": bert["label"],
    "distil": distil["label"],
    "ens": ens["label"]
})

# Majority vote
df["final_pred"], _ = mode(df[["bert","distil","ens"]].values, axis=1)

# Save new ensemble
df[["id","final_pred"]].to_csv("final_ensemble.tsv", sep="\t", index=False)
print("Saved to final_ensemble.tsv")


TypeError: Argument `a` is not recognized as numeric. Support for input that cannot be coerced to a numeric array was deprecated in SciPy 1.9.0 and removed in SciPy 1.11.0. Please consider `np.unique`.

In [59]:

df = pd.DataFrame({
    "id": bert["id"],
    "bert": bert["label"],
    "distil": distil["label"],
    "ens": ens["label"]
})

# majority vote using pandas mode (works with strings)
df["final_pred"] = df[["bert","distil","ens"]].mode(axis=1)[0]

df[["id","final_pred"]].to_csv("final_ensemble.tsv", sep="\t", index=False)
print("Saved to final_ensemble.tsv")


Saved to final_ensemble.tsv


In [60]:
df

,id,bert,distil,ens,final_pred
0,879187,NaN,NaN,NaN,NaN
1,316919,Profane,Profane,Profane,Profane
2,916242,NaN,NaN,NaN,NaN
3,786824,NaN,NaN,NaN,NaN
4,47284,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2507,776466,NaN,NaN,NaN,NaN
2508,849227,Profane,Profane,Profane,Profane
2509,532697,NaN,Profane,Profane,Profane
2510,861411,NaN,NaN,NaN,NaN


In [61]:
df['model']='Ensemble-based Transfromer model'

In [62]:
df

,id,bert,distil,ens,final_pred,model
0,879187,NaN,NaN,NaN,NaN,Ensemble-based Transfromer model
1,316919,Profane,Profane,Profane,Profane,Ensemble-based Transfromer model
2,916242,NaN,NaN,NaN,NaN,Ensemble-based Transfromer model
3,786824,NaN,NaN,NaN,NaN,Ensemble-based Transfromer model
4,47284,NaN,NaN,NaN,NaN,Ensemble-based Transfromer model
...,...,...,...,...,...,...
2507,776466,NaN,NaN,NaN,NaN,Ensemble-based Transfromer model
2508,849227,Profane,Profane,Profane,Profane,Ensemble-based Transfromer model
2509,532697,NaN,Profane,Profane,Profane,Ensemble-based Transfromer model
2510,861411,NaN,NaN,NaN,NaN,Ensemble-based Transfromer model


In [63]:
df

,id,bert,distil,ens,final_pred,model
0,879187,NaN,NaN,NaN,NaN,Ensemble-based Transfromer model
1,316919,Profane,Profane,Profane,Profane,Ensemble-based Transfromer model
2,916242,NaN,NaN,NaN,NaN,Ensemble-based Transfromer model
3,786824,NaN,NaN,NaN,NaN,Ensemble-based Transfromer model
4,47284,NaN,NaN,NaN,NaN,Ensemble-based Transfromer model
...,...,...,...,...,...,...
2507,776466,NaN,NaN,NaN,NaN,Ensemble-based Transfromer model
2508,849227,Profane,Profane,Profane,Profane,Ensemble-based Transfromer model
2509,532697,NaN,Profane,Profane,Profane,Ensemble-based Transfromer model
2510,861411,NaN,NaN,NaN,NaN,Ensemble-based Transfromer model


In [71]:
df["label"] = df["final_pred"].where(pd.notna(df["final_pred"]), 'None')


In [72]:
df

,id,bert,distil,ens,final_pred,model,label
0,879187,NaN,NaN,NaN,None,Ensemble-based Transfromer model,None
1,316919,Profane,Profane,Profane,Profane,Ensemble-based Transfromer model,Profane
2,916242,NaN,NaN,NaN,None,Ensemble-based Transfromer model,None
3,786824,NaN,NaN,NaN,None,Ensemble-based Transfromer model,None
4,47284,NaN,NaN,NaN,None,Ensemble-based Transfromer model,None
...,...,...,...,...,...,...,...
2507,776466,NaN,NaN,NaN,None,Ensemble-based Transfromer model,None
2508,849227,Profane,Profane,Profane,Profane,Ensemble-based Transfromer model,Profane
2509,532697,NaN,Profane,Profane,Profane,Ensemble-based Transfromer model,Profane
2510,861411,NaN,NaN,NaN,None,Ensemble-based Transfromer model,None


In [74]:
df[["id","label", "model"]].to_csv("final_ensemble.tsv", sep="\t", index=False)
print("Saved to final_ensemble.tsv")

Saved to final_ensemble.tsv


In [75]:
df

,id,bert,distil,ens,final_pred,model,label
0,879187,NaN,NaN,NaN,None,Ensemble-based Transfromer model,None
1,316919,Profane,Profane,Profane,Profane,Ensemble-based Transfromer model,Profane
2,916242,NaN,NaN,NaN,None,Ensemble-based Transfromer model,None
3,786824,NaN,NaN,NaN,None,Ensemble-based Transfromer model,None
4,47284,NaN,NaN,NaN,None,Ensemble-based Transfromer model,None
...,...,...,...,...,...,...,...
2507,776466,NaN,NaN,NaN,None,Ensemble-based Transfromer model,None
2508,849227,Profane,Profane,Profane,Profane,Ensemble-based Transfromer model,Profane
2509,532697,NaN,Profane,Profane,Profane,Ensemble-based Transfromer model,Profane
2510,861411,NaN,NaN,NaN,None,Ensemble-based Transfromer model,None
